In [1]:
import pandas as pd
import re

!pip install plotly_express
#importing plotly for creating visualization
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.io as pio

# importing tensorflow and tf_hub for finding similar episodes
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.decomposition import PCA


In [2]:
from google.colab import files
u=files.upload()

Saving the_office_series.csv to the_office_series.csv


In [3]:
episodesData = pd.read_csv('the_office_series.csv')

In [5]:
# This dataset has all the info about the episodes

episodesData.head()

,Unnamed: 0,Season,EpisodeTitle,About,Ratings,Votes,Viewership,Duration,Date,GuestStars,Director,Writers
0,0,1,Pilot,The premiere episode introduces the boss and s...,7.5,4936,11.2,23,24 March 2005,NaN,Ken Kwapis,Ricky Gervais |Stephen Merchant and Greg Daniels
1,1,1,Diversity Day,Michael's off color remark puts a sensitivity ...,8.3,4801,6.0,23,29 March 2005,NaN,Ken Kwapis,B. J. Novak
2,2,1,Health Care,Michael leaves Dwight in charge of picking the...,7.8,4024,5.8,22,5 April 2005,NaN,Ken Whittingham,Paul Lieberstein
3,3,1,The Alliance,"Just for a laugh, Jim agrees to an alliance wi...",8.1,3915,5.4,23,12 April 2005,NaN,Bryan Gordon,Michael Schur
4,4,1,Basketball,Michael and his staff challenge the warehouse ...,8.4,4294,5.0,23,19 April 2005,NaN,Greg Daniels,Greg Daniels


In [4]:
from google.colab import files
v=files.upload()

Saving The-Office-Lines-V3.csv to The-Office-Lines-V3.csv


In [6]:
lines=pd.read_csv('The-Office-Lines-V3.csv',encoding = 'latin-1')
lines.head()

# This dataset has all the info about the lines of characters in the episodes

,season,episode,title,scene,speaker,line
0,1,1,Pilot,1,Michael,All right Jim. Your quarterlies look very good...
1,1,1,Pilot,1,Jim,"Oh, I told you. I couldn't close it. So..."
2,1,1,Pilot,1,Michael,So you've come to the master for guidance? Is ...
3,1,1,Pilot,1,Jim,"Actually, you called me in here, but yeah."
4,1,1,Pilot,1,Michael,"All right. Well, let me show you how it's done."


# Analysis of Episodes

In [7]:
episodesData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    188 non-null    int64  
 1   Season        188 non-null    int64  
 2   EpisodeTitle  188 non-null    object 
 3   About         188 non-null    object 
 4   Ratings       188 non-null    float64
 5   Votes         188 non-null    int64  
 6   Viewership    188 non-null    float64
 7   Duration      188 non-null    int64  
 8   Date          188 non-null    object 
 9   GuestStars    29 non-null     object 
 10  Director      188 non-null    object 
 11  Writers       188 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 17.8+ KB


In [8]:
episodesData.describe()

,Unnamed: 0,Season,Ratings,Votes,Viewership,Duration
count,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000
mean,93.500000,5.468085,8.237234,2838.228723,7.246330,27.053191
std,54.415071,2.386245,0.589930,1063.165290,2.066012,6.937254
min,0.000000,1.000000,6.600000,1832.000000,3.250000,19.000000
25%,46.750000,3.000000,7.800000,2187.750000,5.990000,22.000000
50%,93.500000,6.000000,8.200000,2614.000000,7.535000,23.000000
75%,140.250000,7.250000,8.600000,3144.250000,8.425000,30.000000
max,187.000000,9.000000,9.800000,10515.000000,22.910000,60.000000


In [9]:
episodesData.drop(columns='Unnamed: 0',inplace=True)

The show has been running for 9 seasons. The highest rating for an episode was **9.8** and the lowest was **6.6**. 
Maximum episode duration was 60 minutes and average duration 27 minutes.

In [10]:
episodes_group= episodesData.groupby(['Season'])[['Duration','Ratings']].mean().reset_index()
episodes_group


,Season,Duration,Ratings
0,1,22.833333,7.966667
1,2,22.636364,8.440909
2,3,25.782609,8.586957
3,4,32.500000,8.564286
4,5,28.884615,8.488462
5,6,29.038462,8.196154
6,7,31.083333,8.308333
7,8,23.000000,7.604167
8,9,26.043478,7.913043


In [11]:
fig = px.scatter(episodes_group, x = 'Season', y = 'Ratings', trendline = 'lowess', trendline_options=dict(frac=0.1), size = 'Duration',
                 title = '<b>Ratings over each Season</b>')
fig.show()

# About this trendline : Locally WEighted Scatterplot Smoothing (LOWESS)
# Plotly Express also supports non-linear LOWESS trendlines

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [12]:
fig1 = px.scatter(episodes_group, x = 'Season', y = 'Duration', trendline = 'lowess', trendline_options=dict(frac=0.1), size = 'Ratings',
                 title = '<b>Duration over each Season</b>')
fig1.show()

In [13]:
views= episodesData.groupby(['Season'])[['Viewership','Votes']].mean().reset_index()
views

,Season,Viewership,Votes
0,1,6.366667,4304.000000
1,2,8.168182,3493.909091
2,3,8.494348,3224.086957
3,4,8.547857,3195.642857
4,5,8.756538,2885.615385
5,6,7.766154,2505.307692
6,7,7.305833,2748.083333
7,8,5.385417,2039.250000
8,9,4.138696,2475.826087


In [14]:
fig2= px.scatter(views, x = 'Season', y = 'Votes', trendline = 'lowess',size = 'Viewership',
                 title = '<b>Views and votes over each Season</b>')
fig2.show()

In [15]:
writers=episodesData.groupby(['Writers'])['EpisodeTitle'].count().sort_values(ascending=False).reset_index()
writers

,Writers,EpisodeTitle
0,Mindy Kaling,20
1,B. J. Novak,15
2,Paul Lieberstein,13
3,Greg Daniels,9
4,Brent Forrester,9
5,Justin Spitzer,9
6,Gene Stupnitsky | Lee Eisenberg,8
7,Jennifer Celotta,8
8,Michael Schur,7
9,Charlie Grandy,7


In [16]:
fig3 = px.bar(writers,x = 'EpisodeTitle', y = 'Writers', title = 'Number of Episodes Written by Writers')

fig3.show()

**Mindy Kaling** has writen most of the episodes, which is 20.

In [17]:
episodesData.head()

,Season,EpisodeTitle,About,Ratings,Votes,Viewership,Duration,Date,GuestStars,Director,Writers
0,1,Pilot,The premiere episode introduces the boss and s...,7.5,4936,11.2,23,24 March 2005,NaN,Ken Kwapis,Ricky Gervais |Stephen Merchant and Greg Daniels
1,1,Diversity Day,Michael's off color remark puts a sensitivity ...,8.3,4801,6.0,23,29 March 2005,NaN,Ken Kwapis,B. J. Novak
2,1,Health Care,Michael leaves Dwight in charge of picking the...,7.8,4024,5.8,22,5 April 2005,NaN,Ken Whittingham,Paul Lieberstein
3,1,The Alliance,"Just for a laugh, Jim agrees to an alliance wi...",8.1,3915,5.4,23,12 April 2005,NaN,Bryan Gordon,Michael Schur
4,1,Basketball,Michael and his staff challenge the warehouse ...,8.4,4294,5.0,23,19 April 2005,NaN,Greg Daniels,Greg Daniels


In [41]:
best_ratings= episodesData[['Ratings','EpisodeTitle']].sort_values(by='Ratings',ascending=False)
best_ratings

,Ratings,EpisodeTitle
187,9.8,Finale
137,9.8,"Goodbye, Michael"
77,9.7,Stress Relief
186,9.5,A.A.R.M.
59,9.5,Dinner Party
...,...,...
169,7.0,Here Comes Treble
161,7.0,Angry Andy
148,6.9,Gettysburg
104,6.8,The Banker


**The top-5 episodes with the highest ratings are:** 

Finale (9.8)

Goodbye, Michael (9.8)

Stress Relief (9.7)

A.A.R.M. (9.5)

Dinner Party (9.5)

In [19]:
# https://www.kaggle.com/foolofatook/plotly-the-office-fun-from-home/notebook

castDirectors = ['Paul Lieberstein', 'B.J. Novak','Steve Carell', 'John Krasinski','Rainn Wilson','Mindy Kaling']
fig8 = make_subplots(rows = 3, cols = 2, specs = [[{"type":"table"}] * 2]*3,
                                subplot_titles = castDirectors,
                                horizontal_spacing=0.03,vertical_spacing = 0.07)
for i, director in enumerate(castDirectors):
    df = episodesData[episodesData['Director'] == director].sort_values(by = 'Ratings',ascending = False)
    trace = go.Table(header = dict(values = ['<b>Episode Name</b>','<b>Rating</b>']), cells = dict(values = [df['EpisodeTitle'], df['Ratings']]))
    fig8.add_trace(trace, row = (i//2)+1 , col = (i%2)+1)
fig8.update_layout(height = 500,margin=dict(l=80, r=80, t=100, b=20),
                  title = { 'text' : '<b>Cast Directed Episodes</b>', 'x' : 0.5})
fig8.show()

In [20]:
episodesData['GuestStars'].value_counts()

Amy Adams                                                2
Nancy Carell                                             1
Michael Imperioli                                        1
Ryan Howard                                              1
Julius Winfield Erving                                   1
Josh Groban                                              1
Stephen Colbert                                          1
Chris Bauer                                              1
Maura Tierney                                            1
Josh Groban, Stephen Collins                             1
Will Ferrell                                             1
Will Arnett,  Ray Romano, Jim Carrey, Ricky Gervais      1
Ricky Gervais                                            1
Timothy Olyphant                                         1
Evan Peters                                              1
Mellisa Rauch                                            1
Christian Slater                                        

We will find out which episodes included guest stars.

In [21]:
withguest=episodesData['GuestStars'].notnull()

episodesData['with_guest']= withguest

display(episodesData)

,Season,EpisodeTitle,About,Ratings,Votes,Viewership,Duration,Date,GuestStars,Director,Writers,with_guest
0,1,Pilot,The premiere episode introduces the boss and s...,7.5,4936,11.20,23,24 March 2005,NaN,Ken Kwapis,Ricky Gervais |Stephen Merchant and Greg Daniels,False
1,1,Diversity Day,Michael's off color remark puts a sensitivity ...,8.3,4801,6.00,23,29 March 2005,NaN,Ken Kwapis,B. J. Novak,False
2,1,Health Care,Michael leaves Dwight in charge of picking the...,7.8,4024,5.80,22,5 April 2005,NaN,Ken Whittingham,Paul Lieberstein,False
3,1,The Alliance,"Just for a laugh, Jim agrees to an alliance wi...",8.1,3915,5.40,23,12 April 2005,NaN,Bryan Gordon,Michael Schur,False
4,1,Basketball,Michael and his staff challenge the warehouse ...,8.4,4294,5.00,23,19 April 2005,NaN,Greg Daniels,Greg Daniels,False
...,...,...,...,...,...,...,...,...,...,...,...,...
183,9,Stairmageddon,Dwight shoots Stanley with a bull tranquilizer...,8.0,1985,3.83,22,11 April 2013,NaN,Matt Sohn,Dan Sterling,False
184,9,Paper Airplane,The employees hold a paper airplane competitio...,8.0,2007,3.25,22,25 April 2013,NaN,Jesse Peretz,Halsted Sullivan | Warren Lieberstein,False
185,9,Livin' the Dream,Dwight becomes regional manager after Andy qui...,9.0,2831,3.51,42,2 May 2013,Michael Imperioli,Jeffrey Blitz,Niki Schwartz-Wright,True
186,9,A.A.R.M.,Dwight prepares for a marriage proposal and hi...,9.5,3914,4.56,43,9 May 2013,NaN,David Rogers,Brent Forrester,False


In [22]:
# we will create a dataset where the episodes have guests appearing.

guest_episodes= episodesData[episodesData['with_guest']==True]
guest_episodes

,Season,EpisodeTitle,About,Ratings,Votes,Viewership,Duration,Date,GuestStars,Director,Writers,with_guest
5,1,Hot Girl,Michael is just one of the many male staff who...,7.7,3854,4.80,23,26 April 2005,Amy Adams,Amy Heckerling,Mindy Kaling,True
8,2,Office Olympics,"Ready to finalize his deal for a new condo, Mi...",8.4,3665,8.30,22,4 October 2005,Nancy Carell,Paul Feig,Michael Schur,True
9,2,The Fire,A fire in the kitchen relegates the staff to t...,8.4,3607,7.60,22,11 October 2005,Amy Adams,Ken Kwapis,B. J. Novak,True
12,2,The Client,With Michael and Jan Levinson-Gould away from ...,8.6,3533,7.50,22,8 November 2005,Tim Meadows,Greg Daniels,Paul Lieberstein,True
14,2,E-Mail Surveillance,Michael decides to set up e-mail surveillance ...,8.4,3338,8.10,23,22 November 2005,Ken Jeong,Paul Feig,Jennifer Celotta,True
16,2,Booze Cruise,"The Dunder Mifflin staff go on a mandatory ""bo...",8.6,3536,8.70,22,5 January 2006,Rob Riggle,Ken Kwapis,Greg Daniels,True
21,2,Valentine's Day,When Michael visits Dunder Mifflin corporate h...,8.3,3104,8.95,22,9 February 2006,Conan O'Brien,Greg Daniels,Michael Schur,True
29,3,The Convention,Michael and Dwight go to a convention in Phila...,8.2,3027,7.78,22,28 September 2006,Jerome Bettis,Ken Whittingham,Gene Stupnitsky | Lee Eisenberg,True
53,4,Launch Party,The Dunder Mifflin Infinity website is launchi...,8.4,2844,8.91,42,11 October 2007,Kevin McHale,See full summary,Jennifer Celotta,True
66,5,Business Ethics,A confession at an H.R. business ethics semina...,8.3,2654,8.99,22,9 October 2008,Wendi McLendon-Covey,Jeffrey Blitz,Ryan Koh,True


In [23]:
guest=guest_episodes[['Ratings','Season',	'EpisodeTitle', 'GuestStars']].sort_values(by='Ratings',ascending=False)
guest

,Ratings,Season,EpisodeTitle,GuestStars
187,9.8,9,Finale,"Joan Cusack, Ed Begley Jr, Rachel Harris, Nanc..."
77,9.7,5,Stress Relief,"Cloris Leachman, Jack Black, Jessica Alba"
185,9.0,9,Livin' the Dream,Michael Imperioli
12,8.6,2,The Client,Tim Meadows
16,8.6,2,Booze Cruise,Rob Riggle
130,8.5,7,The Search,"Will Arnett, Ray Romano, Jim Carrey, Ricky Ge..."
108,8.5,6,The Delivery: Part 2,Mellisa Rauch
14,8.4,2,E-Mail Surveillance,Ken Jeong
117,8.4,7,Nepotism,Evan Peters
9,8.4,2,The Fire,Amy Adams


In [24]:
without_guest_episodes= episodesData[episodesData['with_guest']==False]
without_guest_episodes

,Season,EpisodeTitle,About,Ratings,Votes,Viewership,Duration,Date,GuestStars,Director,Writers,with_guest
0,1,Pilot,The premiere episode introduces the boss and s...,7.5,4936,11.20,23,24 March 2005,NaN,Ken Kwapis,Ricky Gervais |Stephen Merchant and Greg Daniels,False
1,1,Diversity Day,Michael's off color remark puts a sensitivity ...,8.3,4801,6.00,23,29 March 2005,NaN,Ken Kwapis,B. J. Novak,False
2,1,Health Care,Michael leaves Dwight in charge of picking the...,7.8,4024,5.80,22,5 April 2005,NaN,Ken Whittingham,Paul Lieberstein,False
3,1,The Alliance,"Just for a laugh, Jim agrees to an alliance wi...",8.1,3915,5.40,23,12 April 2005,NaN,Bryan Gordon,Michael Schur,False
4,1,Basketball,Michael and his staff challenge the warehouse ...,8.4,4294,5.00,23,19 April 2005,NaN,Greg Daniels,Greg Daniels,False
...,...,...,...,...,...,...,...,...,...,...,...,...
180,9,Moving On,"Andy finds out that Erin dumped him for Pete, ...",8.1,2089,4.06,42,14 February 2013,NaN,Jon Favreau,Graham Wagner,False
181,9,The Farm,Show follows Dwight Schrute a beet farmer runn...,7.5,2189,3.54,22,14 March 2013,NaN,Paul Lieberstein,Paul Lieberstein,False
183,9,Stairmageddon,Dwight shoots Stanley with a bull tranquilizer...,8.0,1985,3.83,22,11 April 2013,NaN,Matt Sohn,Dan Sterling,False
184,9,Paper Airplane,The employees hold a paper airplane competitio...,8.0,2007,3.25,22,25 April 2013,NaN,Jesse Peretz,Halsted Sullivan | Warren Lieberstein,False


In [25]:
without= without_guest_episodes[['Ratings','Season',	'EpisodeTitle', 'GuestStars']].sort_values(by='Ratings',ascending=False)
without

,Ratings,Season,EpisodeTitle,GuestStars
137,9.8,7,"Goodbye, Michael",NaN
186,9.5,9,A.A.R.M.,NaN
59,9.5,4,Dinner Party,NaN
27,9.4,2,Casino Night,NaN
132,9.4,7,Threat Level Midnight,NaN
...,...,...,...,...
161,7.0,8,Angry Andy,NaN
162,7.0,8,Fundraiser,NaN
148,6.9,8,Gettysburg,NaN
104,6.8,6,The Banker,NaN


In [27]:
fig9 = px.bar(guest,x = 'EpisodeTitle', y = 'Ratings', title = 'Episodes with guests- Ratings')

fig9.show()

In [28]:
fig10 = px.bar(without,x = 'EpisodeTitle', y = 'Ratings', title = 'Episodes without guests- Ratings')

fig10.show()

In [29]:
guest_episodes['Ratings'].mean()

8.196551724137931

In [30]:
without_guest_episodes['Ratings'].mean()

8.244654088050314

In [31]:
without_guest_episodes[without_guest_episodes['Ratings']>9.0]['EpisodeTitle'].count()

13

In [32]:
guest_episodes[guest_episodes['Ratings']>9.0]['EpisodeTitle'].count()

2

**Conclusion**: 13 of the episodes have ranking above 9.0 and no guest stars appeared,
while only 2 of the episodes with guest stars have ranking above 9.0 and also the mean rating is higher in the episodes without guest stars.

In [48]:
xmas_list=[]

for i in range(0,len(episodesData)):
  x=re.findall('Christmas|christmas|santa|Santa|xmas', episodesData['EpisodeTitle'][i])

  if x:
     xmas_list.append(episodesData.index[i])

for i in xmas_list:
    print(episodesData.EpisodeTitle[i])

Christmas Party
A Benihana Christmas
Moroccan Christmas
Secret Santa
Classy Christmas
Christmas Wishes
Dwight Christmas


In [50]:
words=[]

for i in range(0,len(episodesData)):
  y=re.findall('basketball|Basketball|olympics|Olympics|Sport',episodesData['EpisodeTitle'][i])

  if y:
    words.append(episodesData.index[i])

    for i in words:
      print(episodesData.EpisodeTitle[i])
  


Basketball
Basketball
Office Olympics


In [54]:
michael=[]

for i in range(0,len(episodesData)):
  z=re.findall('Mike|Michael|Scott|scott|Scarn',episodesData['EpisodeTitle'][i])

  if z:
    michael.append(episodesData.index[i])

    for i in michael:
      print(episodesData.EpisodeTitle[i])

Michael's Birthday
Michael's Birthday
Michael Scott Paper Company
Michael's Birthday
Michael Scott Paper Company
Scott's Tots
Michael's Birthday
Michael Scott Paper Company
Scott's Tots
Michael's Last Dundies
Michael's Birthday
Michael Scott Paper Company
Scott's Tots
Michael's Last Dundies
Goodbye, Michael


# Lines Dataset Analysis

In [33]:
lines.head()

,season,episode,title,scene,speaker,line
0,1,1,Pilot,1,Michael,All right Jim. Your quarterlies look very good...
1,1,1,Pilot,1,Jim,"Oh, I told you. I couldn't close it. So..."
2,1,1,Pilot,1,Michael,So you've come to the master for guidance? Is ...
3,1,1,Pilot,1,Jim,"Actually, you called me in here, but yeah."
4,1,1,Pilot,1,Michael,"All right. Well, let me show you how it's done."


In [34]:
print(f"There are {lines['speaker'].nunique()} unique speakers in the show")

There are 775 unique speakers in the show


In [35]:
numberOfLinesSpoken = lines['speaker'].value_counts().reset_index()
numberOfLinesSpoken

,index,speaker
0,Michael,10773
1,Dwight,6752
2,Jim,6222
3,Pam,4973
4,Andy,3698
...,...,...
770,Fake Stanley,1
771,Michael and Erin,1
772,Deliveryman,1
773,Zion,1


In [36]:
numberOfLinesSpoken = lines['speaker'].value_counts().reset_index()
numberOfLinesSpoken.columns = ['Speaker','Number of Dialogues']
numberOfLinesSpoken = numberOfLinesSpoken.sort_values(by = 'Number of Dialogues', ascending = False)
fig = px.bar(numberOfLinesSpoken[:10][::-1],x = 'Number of Dialogues', y = 'Speaker', orientation = 'h', 
             title = '<b>Top 10 Speakers with the Most Lines</b>',
            color_continuous_scale=px.colors.sequential.Blugrn,color = 'Number of Dialogues',text = 'Number of Dialogues')
fig.show()

Micahel leads in the speakers in the show with a big difference from the second one, and the amazing/weird fact is that he stopps appearing after season 7.

In [37]:


fig = make_subplots(rows = 3,cols = 3, 
                    subplot_titles=[f'Season {i}' for i in range(1,10)],
                   horizontal_spacing=0.1)

for i in range(3):
    for j in range(3):
        season = i*3 + j + 1
        seasonDf = lines[lines['season'] == season]
        speakerDialogues = seasonDf['speaker'].value_counts().reset_index()
        speakerDialogues.columns = ['Speaker','Number of Dialogues']
        speakerDialogues = speakerDialogues.sort_values(by = 'Number of Dialogues', ascending = False).iloc[:3,]
        trace = go.Bar(x = speakerDialogues['Number of Dialogues'], y = speakerDialogues['Speaker'], name = f'Season {season}', orientation = 'h')
        fig.add_trace(trace, row = i+1, col = j+1)
fig.update_layout(showlegend = False, title = '<b>Top 3 Speakers each Season</b>')
fig.show()

In [38]:
def formatLine(line):
    line = line.lower()
    # replace whatever is not a letter with space
    line = re.sub(r'[^\w\s]','',line)
    return line

lines['formatted_lines'] = lines['line'].apply(lambda x:formatLine(x))

In [39]:
df = lines[lines['formatted_lines'].str.contains("thats what she said")]
df = df['speaker'].value_counts().reset_index()[:5]
df.columns = ['Speaker', 'Number of References']
df

,Speaker,Number of References
0,Michael,23
1,Dwight,3
2,Jim,2
3,Jan,1
4,Deposition Reporter,1


In [56]:
# Number of times Dwight said Michael

dwight= lines[lines['speaker']=='Dwight'].reset_index()
dwight

,index,season,episode,title,scene,speaker,line,formatted_lines
0,16,1,1,Pilot,5,Dwight,Shall I play for you? Pa rum pump um pum I h...,shall i play for you pa rum pump um pum i ha...
1,21,1,1,Pilot,7,Dwight,Whassup!,whassup
2,23,1,1,Pilot,7,Dwight,Whassup.,whassup
3,85,1,1,Pilot,19,Dwight,What?,what
4,87,1,1,Pilot,19,Dwight,Just clearing my desk. I can't concentrate.,just clearing my desk i cant concentrate
...,...,...,...,...,...,...,...,...
6747,54582,9,24,Finale,8138,Dwight,"Aww, J, P and D. The Three Amigos. Ah, it warm...",aww j p and d the three amigos ah it warms my ...
6748,54590,9,24,Finale,8140,Dwight,"No, don't say it. You're fired! You're both fi...",no dont say it youre fired youre both fired
6749,54592,9,24,Finale,8140,Dwight,Don't be an idiot. It's for the severance. The...,dont be an idiot its for the severance the bes...
6750,54595,9,24,Finale,8140,Dwight,"Woah. Right. For what, the art? The music? The...",woah right for what the art the music the incr...


In [66]:
count = 0

for i in range(0,len(dwight)):
  x = re.search('Michael',dwight['line'][i])
  if type(x)==re.Match:
   count=count+1
print('Dwight said Michael ',count, ' times.')   

Dwight said Michael  262  times.


In [70]:
count1 = 0
jim=lines[lines['speaker']=='Jim'].reset_index()
for i in range(0,len(jim)):
  y = re.search('Pam', jim['line'][i])
  if type(y)==re.Match:
   count1=count1 + 1
print('Jim said Pam\'s name ',count1, ' times.') 

Jim said Pam's name  193  times.


Number of times that the following characters used **"That's what she said"**

In [91]:
def twss_count(lines,char):
    twsscount = 0

    lines = lines[lines['speaker']==char].reset_index()

    for i in range(0,len(lines)):
        x = re.search('that\'s what she said',lines['line'][i])
        y = re.search('That\'s what she said',lines['line'][i])
        if type(x)==re.Match:
            twsscount = twsscount + 1
        if type(y)==re.Match:
            twsscount = twsscount + 1
    return twsscount

chars = ['Pam','Dwight','Jim','Michael']

tws = []

for i in chars:
    tws.append(twss_count(lines,i))

print(chars, tws)

['Pam', 'Dwight', 'Jim', 'Michael'] [1, 3, 1, 21]
